In [1]:
import os
import time
import sys
import numpy as np
import pandas as pd
import regex as re
import lxml
import numbers
from bs4 import BeautifulSoup
import requests

In [2]:
headers1 = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}

In [3]:
base_url = "https://www.zillow.com/homes/for_sale/"

In [47]:
url_list = []

city = 'bergen county'
url1 = base_url +city+'/'
url_list.append(url1)

for i in range(2,50):
    dom = base_url + city +'/'+str(i)+'_p/'
    url_list.append(dom)

In [48]:
url_list

['https://www.zillow.com/homes/for_sale/bergen county/',
 'https://www.zillow.com/homes/for_sale/bergen county/2_p/',
 'https://www.zillow.com/homes/for_sale/bergen county/3_p/',
 'https://www.zillow.com/homes/for_sale/bergen county/4_p/',
 'https://www.zillow.com/homes/for_sale/bergen county/5_p/',
 'https://www.zillow.com/homes/for_sale/bergen county/6_p/',
 'https://www.zillow.com/homes/for_sale/bergen county/7_p/',
 'https://www.zillow.com/homes/for_sale/bergen county/8_p/',
 'https://www.zillow.com/homes/for_sale/bergen county/9_p/',
 'https://www.zillow.com/homes/for_sale/bergen county/10_p/',
 'https://www.zillow.com/homes/for_sale/bergen county/11_p/',
 'https://www.zillow.com/homes/for_sale/bergen county/12_p/',
 'https://www.zillow.com/homes/for_sale/bergen county/13_p/',
 'https://www.zillow.com/homes/for_sale/bergen county/14_p/',
 'https://www.zillow.com/homes/for_sale/bergen county/15_p/',
 'https://www.zillow.com/homes/for_sale/bergen county/16_p/',
 'https://www.zillow.

In [49]:
dom

'https://www.zillow.com/homes/for_sale/bergen county/49_p/'

In [50]:
def soups(data):
    with requests.Session() as s:
        r = s.get(data, headers=headers1)
        soup = BeautifulSoup(r.content, 'html.parser')
    return soup

In [51]:
list_links = []

for url in url_list:
    htmls = soups(url)
    list_links.append(htmls)
print(len(list_links))

49


In [52]:
df = pd.DataFrame()

In [53]:
def z_soup(data):
    address = data.find_all(class_= 'list-card-addr')
    price = list(data.find_all(class_='list-card-price'))
    beds = list(data.find_all("ul", class_="list-card-details"))
    df['prices'] = price
    df['address'] = address
    df['beds'] = beds
    
    return df.copy()

In [54]:
zillow_df = pd.DataFrame()

In [55]:
df_list = []
for soup in list_links:
    try:
        new_df = z_soup(soup)
        df_list.append(new_df)
    except:
        continue

In [56]:
# Combine the list of datasets into our new dataframe
zillow_df = pd.concat(df_list)
zillow_df.reset_index(inplace=True)
zillow_df = zillow_df.drop('index', axis=1)

In [57]:
print(zillow_df.shape)
zillow_df.head()

(225, 3)


,prices,address,beds
0,"[$725,000]","[231 Kociemba Dr, River Vale, NJ 07675]","[[4, [ , , bds]], [2, [ , , ba]], [--, [ , ..."
1,"[$199,000]","[2 Horizon UNIT G20, Fort Lee Boro, NJ 07024]","[[3, [ , , bds]], [3, [ , , ba]], [--, [ , ..."
2,"[$1,449,000]","[115 Demarest Ave, Englewood Cliffs, NJ 07632]","[[4, [ , , bds]], [3, [ , , ba]], [--, [ , ..."
3,"[$525,000]","[456 Linwood Ave, Ridgewood Village, NJ 07450-...","[[4, [ , , bds]], [2, [ , , ba]], [--, [ , ..."
4,"[$578,777]","[120 Harding Ave, Westwood, NJ 07675]","[[4, [ , , bds]], [2, [ , , ba]], [--, [ , ..."


In [58]:
zillow_df = zillow_df.applymap(str)

In [59]:
zillow_df.shape

(225, 3)

In [60]:
zillow_df = zillow_df.applymap(lambda x: re.sub('<[^<]+?>', '',x))
zillow_df.head()

,prices,address,beds
0,"$725,000","231 Kociemba Dr, River Vale, NJ 07675",4 bds2 ba-- sqft- House for sale
1,"$199,000","2 Horizon UNIT G20, Fort Lee Boro, NJ 07024",3 bds3 ba-- sqft- Condo for sale
2,"$1,449,000","115 Demarest Ave, Englewood Cliffs, NJ 07632",4 bds3 ba-- sqft- House for sale
3,"$525,000","456 Linwood Ave, Ridgewood Village, NJ 07450-3420",4 bds2 ba-- sqft- House for sale
4,"$578,777","120 Harding Ave, Westwood, NJ 07675",4 bds2 ba-- sqft- House for sale


In [61]:
zillow_df[['beds', 'home_type']] = zillow_df.beds.str.split('-',n=1, expand=True)
zillow_df.head()

,prices,address,beds,home_type
0,"$725,000","231 Kociemba Dr, River Vale, NJ 07675",4 bds2 ba,- sqft- House for sale
1,"$199,000","2 Horizon UNIT G20, Fort Lee Boro, NJ 07024",3 bds3 ba,- sqft- Condo for sale
2,"$1,449,000","115 Demarest Ave, Englewood Cliffs, NJ 07632",4 bds3 ba,- sqft- House for sale
3,"$525,000","456 Linwood Ave, Ridgewood Village, NJ 07450-3420",4 bds2 ba,- sqft- House for sale
4,"$578,777","120 Harding Ave, Westwood, NJ 07675",4 bds2 ba,- sqft- House for sale


In [62]:
zillow_df[['address', 'borough','zip_code']] = zillow_df.address.str.split(",", expand=True)
zillow_df.head()

,prices,address,beds,home_type,borough,zip_code
0,"$725,000",231 Kociemba Dr,4 bds2 ba,- sqft- House for sale,River Vale,NJ 07675
1,"$199,000",2 Horizon UNIT G20,3 bds3 ba,- sqft- Condo for sale,Fort Lee Boro,NJ 07024
2,"$1,449,000",115 Demarest Ave,4 bds3 ba,- sqft- House for sale,Englewood Cliffs,NJ 07632
3,"$525,000",456 Linwood Ave,4 bds2 ba,- sqft- House for sale,Ridgewood Village,NJ 07450-3420
4,"$578,777",120 Harding Ave,4 bds2 ba,- sqft- House for sale,Westwood,NJ 07675


In [63]:
# separate beds column into bed, bath, and sq_feet

zillow_df[['beds', 'baths', 'sqft']] = zillow_df.beds.str.split(' ',n=2, expand=True)
zillow_df['sqft'] = zillow_df.sqft.str.replace(",", "")

In [64]:
# extract only the digits from the columns
zillow_df['beds'] = zillow_df.beds.str.extract('(\d+)')
zillow_df['baths'] = zillow_df.baths.str.extract('(\d+)')
zillow_df['sqft'] = zillow_df.sqft.str.extract('(\d+)')

In [65]:
# convert columns to float
zillow_df['beds'] = zillow_df['beds'].astype('float')
zillow_df['baths'] = zillow_df['baths'].astype('float')
zillow_df['sqft'] = zillow_df['sqft'].astype('float')

In [66]:
zillow_df.tail()

,prices,address,beds,home_type,borough,zip_code,baths,sqft
220,"$578,777",120 Harding Ave,4.0,- sqft- House for sale,Westwood,NJ 07675,2.0,NaN
221,"$260,000",44 S Main St APT 3C,2.0,- sqft- Condo for sale,Lodi,NJ 07644,2.0,NaN
222,"$269,000",1085 Stasia St,3.0,- sqft- House for sale,Teaneck,NJ 07666,2.0,NaN
223,"$499,000",80 Iroquois Ave,3.0,- sqft- House for sale,Oakland Boro,NJ 07436-3744,3.0,NaN
224,"$1,195,000",366 Beveridge Rd,5.0,- sqft- House for sale,Ridgewood Village,NJ 07450-1012,3.0,NaN


In [67]:
zillow_df.to_csv("bergen_county_zillow.csv")